# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fd9282e1d90>
├── 'a' --> tensor([[ 0.8606,  1.7264, -1.4719],
│                   [ 1.8389,  0.3471,  0.2538]])
└── 'x' --> <FastTreeValue 0x7fd9282e1d60>
    └── 'c' --> tensor([[ 0.0093, -0.6375,  0.5757,  0.1860],
                        [-0.3042,  0.2066, -0.1245, -0.7442],
                        [-0.9315,  1.9697, -0.0161, -0.5330]])

In [4]:
t.a

tensor([[ 0.8606,  1.7264, -1.4719],
        [ 1.8389,  0.3471,  0.2538]])

In [5]:
%timeit t.a

63.4 ns ± 0.049 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fd9282e1d90>
├── 'a' --> tensor([[-1.5508, -0.7956,  0.0985],
│                   [ 0.1440, -1.6534,  0.2893]])
└── 'x' --> <FastTreeValue 0x7fd9282e1d60>
    └── 'c' --> tensor([[ 0.0093, -0.6375,  0.5757,  0.1860],
                        [-0.3042,  0.2066, -0.1245, -0.7442],
                        [-0.9315,  1.9697, -0.0161, -0.5330]])

In [7]:
%timeit t.a = new_value

64.2 ns ± 0.0315 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.8606,  1.7264, -1.4719],
               [ 1.8389,  0.3471,  0.2538]]),
    x: Batch(
           c: tensor([[ 0.0093, -0.6375,  0.5757,  0.1860],
                      [-0.3042,  0.2066, -0.1245, -0.7442],
                      [-0.9315,  1.9697, -0.0161, -0.5330]]),
       ),
)

In [10]:
b.a

tensor([[ 0.8606,  1.7264, -1.4719],
        [ 1.8389,  0.3471,  0.2538]])

In [11]:
%timeit b.a

53.7 ns ± 0.0537 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.5344, -0.8253,  0.4788],
               [ 0.5287, -0.8002, -1.0607]]),
    x: Batch(
           c: tensor([[ 0.0093, -0.6375,  0.5757,  0.1860],
                      [-0.3042,  0.2066, -0.1245, -0.7442],
                      [-0.9315,  1.9697, -0.0161, -0.5330]]),
       ),
)

In [13]:
%timeit b.a = new_value

478 ns ± 0.0905 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

832 ns ± 0.244 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.2 µs ± 68.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 613 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 1.38 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fd9282daa30>
├── 'a' --> tensor([[[ 0.8606,  1.7264, -1.4719],
│                    [ 1.8389,  0.3471,  0.2538]],
│           
│                   [[ 0.8606,  1.7264, -1.4719],
│                    [ 1.8389,  0.3471,  0.2538]],
│           
│                   [[ 0.8606,  1.7264, -1.4719],
│                    [ 1.8389,  0.3471,  0.2538]],
│           
│                   [[ 0.8606,  1.7264, -1.4719],
│                    [ 1.8389,  0.3471,  0.2538]],
│           
│                   [[ 0.8606,  1.7264, -1.4719],
│                    [ 1.8389,  0.3471,  0.2538]],
│           
│                   [[ 0.8606,  1.7264, -1.4719],
│                    [ 1.8389,  0.3471,  0.2538]],
│           
│                   [[ 0.8606,  1.7264, -1.4719],
│                    [ 1.8389,  0.3471,  0.2538]],
│           
│                   [[ 0.8606,  1.7264, -1.4719],
│                    [ 1.8389,  0.3471,  0.2538]]])
└── 'x' --> <FastTreeValue 0x7fd9282dab50>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.5 µs ± 116 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fd9282d4220>
├── 'a' --> tensor([[ 0.8606,  1.7264, -1.4719],
│                   [ 1.8389,  0.3471,  0.2538],
│                   [ 0.8606,  1.7264, -1.4719],
│                   [ 1.8389,  0.3471,  0.2538],
│                   [ 0.8606,  1.7264, -1.4719],
│                   [ 1.8389,  0.3471,  0.2538],
│                   [ 0.8606,  1.7264, -1.4719],
│                   [ 1.8389,  0.3471,  0.2538],
│                   [ 0.8606,  1.7264, -1.4719],
│                   [ 1.8389,  0.3471,  0.2538],
│                   [ 0.8606,  1.7264, -1.4719],
│                   [ 1.8389,  0.3471,  0.2538],
│                   [ 0.8606,  1.7264, -1.4719],
│                   [ 1.8389,  0.3471,  0.2538],
│                   [ 0.8606,  1.7264, -1.4719],
│                   [ 1.8389,  0.3471,  0.2538]])
└── 'x' --> <FastTreeValue 0x7fd871543a90>
    └── 'c' --> tensor([[ 0.0093, -0.6375,  0.5757,  0.1860],
                        [-0.3042,  0.2066, -0.1245, -0.7442],
                 

In [23]:
%timeit t_cat(trees)

29.7 µs ± 96.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.2 µs ± 131 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.0093, -0.6375,  0.5757,  0.1860],
                       [-0.3042,  0.2066, -0.1245, -0.7442],
                       [-0.9315,  1.9697, -0.0161, -0.5330]],
              
                      [[ 0.0093, -0.6375,  0.5757,  0.1860],
                       [-0.3042,  0.2066, -0.1245, -0.7442],
                       [-0.9315,  1.9697, -0.0161, -0.5330]],
              
                      [[ 0.0093, -0.6375,  0.5757,  0.1860],
                       [-0.3042,  0.2066, -0.1245, -0.7442],
                       [-0.9315,  1.9697, -0.0161, -0.5330]],
              
                      [[ 0.0093, -0.6375,  0.5757,  0.1860],
                       [-0.3042,  0.2066, -0.1245, -0.7442],
                       [-0.9315,  1.9697, -0.0161, -0.5330]],
              
                      [[ 0.0093, -0.6375,  0.5757,  0.1860],
                       [-0.3042,  0.2066, -0.1245, -0.7442],
                       [-0.9315,  1.9697, -0.0161, -0.5330]],

In [26]:
%timeit Batch.stack(batches)

78 µs ± 181 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.0093, -0.6375,  0.5757,  0.1860],
                      [-0.3042,  0.2066, -0.1245, -0.7442],
                      [-0.9315,  1.9697, -0.0161, -0.5330],
                      [ 0.0093, -0.6375,  0.5757,  0.1860],
                      [-0.3042,  0.2066, -0.1245, -0.7442],
                      [-0.9315,  1.9697, -0.0161, -0.5330],
                      [ 0.0093, -0.6375,  0.5757,  0.1860],
                      [-0.3042,  0.2066, -0.1245, -0.7442],
                      [-0.9315,  1.9697, -0.0161, -0.5330],
                      [ 0.0093, -0.6375,  0.5757,  0.1860],
                      [-0.3042,  0.2066, -0.1245, -0.7442],
                      [-0.9315,  1.9697, -0.0161, -0.5330],
                      [ 0.0093, -0.6375,  0.5757,  0.1860],
                      [-0.3042,  0.2066, -0.1245, -0.7442],
                      [-0.9315,  1.9697, -0.0161, -0.5330],
                      [ 0.0093, -0.6375,  0.5757,  0.1860],
                   

In [28]:
%timeit Batch.cat(batches)

143 µs ± 363 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

310 µs ± 7.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
